<a href="https://colab.research.google.com/github/ramesitexp/genai_usecase/blob/main/ResumeClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 84.7 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade accelerate -q
!pip uninstall -y transformers accelerate -q
!pip install transformers accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 22.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/content/Resume.csv')
data.head()

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [ ]:
data.shape

(2484, 4)

In [ ]:
your_num_labels = data['Category'].nunique()
your_num_labels

24

In [ ]:
from sklearn.model_selection import train_test_split

data = data[['Resume_str', 'Category']].rename(columns={'Resume_str': 'resume_str', 'Category': 'category'})

# Split the data into train and temporary sets (80-20)
train_data, temp_data = train_test_split(data, test_size=0.2, random_state=42)

# Split the temporary set into test and validation sets (50-50)
test_data, validation_data = train_test_split(temp_data, test_size=0.5, random_state=42)


In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)
validation_dataset = Dataset.from_pandas(validation_data)


In [ ]:
train_dataset[0]

{'resume_str': "         GRADUATE ENGINEER       Summary    Intellectually curious and resourceful\n*Great interpersonal skills and ability to communicate effectively at all levels within an organization        Experience     03/2014   to   Current     Graduate Engineer    Company Name   －   City  ,   State      Developed conceptual plans for two intersections near Jersey Shore University Medical Center, based on traffic data, geometric constraints, and roadway design standards.  Employed knowledge of Microstation, Inroads, Excel, and VBA to efficiently complete miscellaneous tasks, including producing reports, creating visual displays, and generating automated text descriptions.  Applied vehicle turning software (AutoTurn) to analyze intersection design and geometric layout on various projects.  Member of the highway design team on I-295 Direct Connection Contract 3 project; developed grading on complex pavement widening sections, revised vertical and horizontal alignments to meet NJD

In [ ]:
from datasets import DatasetDict

dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'validation': validation_dataset
})


In [ ]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['resume_str', 'category', '__index_level_0__'],
        num_rows: 1987
    })
    test: Dataset({
        features: ['resume_str', 'category', '__index_level_0__'],
        num_rows: 248
    })
    validation: Dataset({
        features: ['resume_str', 'category', '__index_level_0__'],
        num_rows: 249
    })
})

In [ ]:
!pip install huggingface_hub -q

In [ ]:
from google.colab import userdata
HUGGINGFACE_TOKEN = userdata.get('HUGGINGFACE_TOKEN')

In [ ]:
!huggingface-cli login --token $HUGGINGFACE_TOKEN

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from datasets import DatasetDict

dataset_dict.push_to_hub('resume_dataset')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/pallavi176/resume_dataset/commit/f0efda7d24f9c1b216e42367de264c85cc051f8c', commit_message='Upload dataset', commit_description='', oid='f0efda7d24f9c1b216e42367de264c85cc051f8c', pr_url=None, pr_revision=None, pr_num=None)

# After uploading dataset to the Hub, start from here

In [ ]:
from datasets import load_dataset, load_from_disk, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
# https://huggingface.co/datasets/pallavi176/resume_dataset
# Step 1: Load dataset from hub
data = load_dataset("pallavi176/resume_dataset")
data

DatasetDict({
    train: Dataset({
        features: ['resume_str', 'category', '__index_level_0__'],
        num_rows: 1987
    })
    test: Dataset({
        features: ['resume_str', 'category', '__index_level_0__'],
        num_rows: 248
    })
    validation: Dataset({
        features: ['resume_str', 'category', '__index_level_0__'],
        num_rows: 249
    })
})

In [ ]:
data['train']

Dataset({
    features: ['resume_str', 'category', '__index_level_0__'],
    num_rows: 1987
})

In [ ]:
#Step 2: Load a tokenizer and model from the Hugging Face Model Hub
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Create label mappings
unique_labels = set(data['train']['category'] + data['test']['category'] + data['validation']['category'])
label2id = {label: id for id, label in enumerate(unique_labels)}
id2label = {id: label for label, id in label2id.items()}

In [ ]:
# Step 3: Tokenize the dataset
def tokenize_and_encode(ds):
    tokenized = tokenizer(ds['resume_str'], padding='max_length', truncation=True, max_length=512)
    tokenized['labels'] = [label2id[label] for label in ds['category']]
    return tokenized

tokenized_datasets = data.map(tokenize_and_encode, batched=True)

Map:   0%|          | 0/1987 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

Map:   0%|          | 0/249 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['resume_str', 'category', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1987
    })
    test: Dataset({
        features: ['resume_str', 'category', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 248
    })
    validation: Dataset({
        features: ['resume_str', 'category', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 249
    })
})

In [ ]:
tokenized_datasets['train']['resume_str'][0]

"         GRADUATE ENGINEER       Summary    Intellectually curious and resourceful\n*Great interpersonal skills and ability to communicate effectively at all levels within an organization        Experience     03/2014   to   Current     Graduate Engineer    Company Name   －   City  ,   State      Developed conceptual plans for two intersections near Jersey Shore University Medical Center, based on traffic data, geometric constraints, and roadway design standards.  Employed knowledge of Microstation, Inroads, Excel, and VBA to efficiently complete miscellaneous tasks, including producing reports, creating visual displays, and generating automated text descriptions.  Applied vehicle turning software (AutoTurn) to analyze intersection design and geometric layout on various projects.  Member of the highway design team on I-295 Direct Connection Contract 3 project; developed grading on complex pavement widening sections, revised vertical and horizontal alignments to meet NJDOT standards, m

In [ ]:
tokenized_datasets['train']['category'][0]

'AVIATION'

In [ ]:
tokenized_datasets['train']['labels'][0]

8

In [ ]:
# Load the model with the custom label configuration
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=len(unique_labels), id2label=id2label, label2id=label2id)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Step 4: Fine-tune the model
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.567369,0.867470,0.824329,0.819936,0.835338


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=497, training_loss=0.6089032977159834, metrics={'train_runtime': 4563.7494, 'train_samples_per_second': 0.435, 'train_steps_per_second': 0.109, 'total_flos': 263315989684224.0, 'train_loss': 0.6089032977159834, 'epoch': 1.0})

In [ ]:
# Save the model and tokenizer
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model_tokens')

('./fine_tuned_model_tokens/tokenizer_config.json',
 './fine_tuned_model_tokens/special_tokens_map.json',
 './fine_tuned_model_tokens/vocab.txt',
 './fine_tuned_model_tokens/added_tokens.json',
 './fine_tuned_model_tokens/tokenizer.json')

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()
eval_results

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.5673694014549255,
 'eval_accuracy': 0.8674698795180723,
 'eval_f1': 0.8243285046620692,
 'eval_precision': 0.8199359818925036,
 'eval_recall': 0.8353379712075365,
 'eval_runtime': 164.7053,
 'eval_samples_per_second': 1.512,
 'eval_steps_per_second': 0.383,
 'epoch': 1.0}